- performance evaluation
- date: 2020-08-06
- maintainer: YZK

In [1]:
# jupyter nbconvert --to script perfeval.ipynb

In [2]:
import copy
import gc
import itertools
import logging
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime
from scipy.stats import norm
from scipy import stats as stats
from sklearn import preprocessing, metrics
from sklearn.metrics import mean_squared_error

try:
    thisd = os.path.dirname(os.path.realpath(__file__))
    sys.path.append(thisd)
    import widgets as wg
    from textloader import textloader 
except Exception as E:
    logging.warning(E)
    if __name__ == "__main__":
        import msetup
        import widgets as wg
        from textloader import textloader    
    else:
        import widgets as wg
        from lib.textloader import textloader
        
plt.style.use('ggplot')
# plt.style.use('seaborn')
# plt.style.use('classic')


<font size=5>$\textit{Precision}=\dfrac{\textit{True Positive}}{\textit{True Positive}\:+\:\textit{False Positive}}$</font>

<font size=5>$\textit{Recall}=\dfrac{\textit{True Positive}}{\textit{True Positive}\:+\:\textit{False Negative}}$</font>

<font size=5>$\textit{F1-score}=2\cdot\dfrac{\textit{precision}\:\times\:\textit{recall}}{\textit{precision}\:+\:\textit{recall}}$</font>

In [ ]:
def plot_confusion_matrix(cm, 
                          classes,
                          fig=None,
                          ax=None,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        cm: confusion matrix whose indexes are true labels and columns are predicted labels
    """
    
    cm_ = np.copy(cm)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # np.newaxis: increse one dimension
        logging.info("Normalized confusion matrix")
    else:
        logging.info('Confusion matrix, without normalization')

    if fig is None and ax is None:
        fig, ax = plt.subplots(figsize=(16, 10))

    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    fig.colorbar(im, ax=ax)
    
    tick_marks = np.arange(len(classes))
    ax.xaxis.tick_top()
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(classes, rotation=45, fontsize=12)  
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(classes, fontsize=12)
    ax.grid(b=False)
    
#     fmt = '.2f' if normalize else 'd'
#     fmt = '{0:.2f}\n({1:d})' if normalize else 'd'
    fmt = '{0:.2f}' if normalize else '{0:d}'

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         ax.text(j, i, format(cm[i, j], fmt),        
#         v = fmt.format(cm[i, j], cm_[i, j]) if normalize else fmt.format(cm[i, j])
        v = fmt.format(cm[i, j])

        ax.text(j, i, v,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize=10)

    ax.set_title(title, fontsize=12)
    ax.set_xlabel('Predicted label', fontsize=12)
    ax.set_ylabel('True label', fontsize=12)
#     fig.tight_layout()
 
    gc.collect()

In [ ]:
def display_table(df, ax, dtype=None, onlytable=False, treverse=False, tloc="bottom", tscale=(1, 2), offset=0.3, bar_width=0.4, title=None):

    columns = df.columns.tolist()
    nclass = len(columns)

    if dtype == "cm":
        TP  = np.zeros(nclass)
        TN  = np.zeros(nclass)
        FP  = np.zeros(nclass)
        FN  = np.zeros(nclass)
        arr = df.values
        for i in range(nclass):
            TP[i] = arr[i, i]
            TN[i] = np.delete(np.delete(arr, i, axis=0), i, axis=1).sum()
            FP[i] = np.delete(arr, i, axis=0)[:, i].sum()
            FN[i] = np.delete(arr, i, axis=1)[i, :].sum()

        rows = ["True Positive", "True Nagetive", "False Positive", "False Negative"]
        data = np.array([TP, TN, FP, FN])
    else:
        rows = df.index.tolist()
        data = df.values

    nrow   = len(rows)
    colors = plt.cm.BuPu(np.linspace(0, 0.5, nrow))
    index  = np.arange(nclass) + offset

    # Initialize the vertical-offset for the stacked bar chart.
    y_offset = np.zeros(nclass)

    # Plot bars and create text labels for the table
    celltext = []
    for row in range(nrow):
        if not onlytable:
            ax.bar(index, data[row, :], bar_width, bottom=y_offset, color=colors[row])
        y_offset = y_offset + data[row, :]
    #     celltext.append(["%1.1f" % x for x in y_offset])
        if dtype == "cm":
            celltext.append(["%d" % data[row, j] for j in range(nclass)])
        else:
            celltext.append(["%.3f" % data[row, j] for j in range(nclass)])


    # Reverse colors and text labels to display the last value at the top.
    if not treverse:
        colors = colors[::-1]
        rows.reverse()
        celltext.reverse()

    if onlytable:
        ax.axis("off")

    # Add a table at the bottom of the axes
    table = ax.table(cellText=celltext, 
                     rowLabels=rows,
                     rowColours=colors,
                     colLabels=columns, 
                     loc=tloc)

    table.set_fontsize(14)
    table.scale(tscale[0], tscale[1])

    # Adjust layout to make room for the table:
    plt.subplots_adjust(left=0.1, bottom=0.2)

#     plt.ylabel("")
    # plt.yticks(values * value_increment, ['%d' % val for val in values])
    ax.set_xticks([])
    if title is not None:
        ax.set_title(title, fontsize=12)

#     plt.show()
    gc.collect()

In [ ]:
def getClasses(endpoints):
    nclass = len(endpoints) + 1
    classes = []
    for idx, val in enumerate(endpoints):
        if idx == 0:
            classes.append("$\gamma\leq{}$".format(val))
            if len(endpoints) - 1 == 0:
                classes.append("${}<\gamma$".format(val))
        elif idx == len(endpoints) - 1:
            classes.append("${}<\gamma\leq{}$".format(endpoints[idx - 1], val)) 
            classes.append("${}<\gamma$".format(val))
        else:
            classes.append("${}<\gamma\leq{}$".format(endpoints[idx - 1], val))
            
    return classes

In [ ]:
class perfeval:
    
    @staticmethod
    def tspredict(y_true, y_pred, outd=None, **kwarg):

        if not isinstance(y_true, np.ndarray):
            y_true = np.array(y_true)

        if not isinstance(y_pred, np.ndarray):
            y_pred = np.array(y_pred)
            
        assert y_true.shape[0] == y_pred.shape[0]

        title_ = None
        if "title" in kwarg.keys():
            title_ = kwarg["title"]

        if np.isnan(y_true).all() or len(y_true[~np.isnan(y_true)]) <= 5:
            outf = "{}/perfeval.png".format(outd)
            if title_ is not None:
                outf = "{}/perfeval_{}.png".format(outd, title_)
            logging.warning("output failure {} (missing y_true).".format(outf))
            return -1
        
        ###  config
        props = dict(boxstyle="round", facecolor="red", alpha=0.5)
        
#         if "y_class_true" in kwarg.keys() and "y_class_pred" in kwarg.keys():
#             cm = metrics.confusion_matrix(kwarg["y_class_true"], kwarg["y_class_pred"])

        if "classes" in kwarg.keys():
            classes = kwarg["classes"]
        elif "classify" in kwarg.keys():
            endpoints = copy.copy(kwarg["classify"])
            nclass = len(endpoints) + 1
            classes = getClasses(endpoints)
            
            logging.info("classes = {}".format(classes))
            
        xposi_ = None
        if "xposi" in kwarg.keys():
            xposi_ = kwarg["xposi"]

        xlabel_ = None
        if "xlabel" in kwarg.keys():
            xlabel_ = kwarg["xlabel"]

#         title_ = None
#         if "title" in kwarg.keys():
#             title_ = kwarg["title"]

        toffset = 0
        if "toffset" in kwarg.keys():
            toffset = kwarg["toffset"]
        
        if "task" in kwarg.keys() and kwarg["task"] == "classification":
            
            y_class = np.unique(np.append(np.unique(y_true), np.unique(y_pred)))
            classes = [classes[i] for i in y_class]
            
            cm = metrics.confusion_matrix(y_true, y_pred)
            cmdf = pd.DataFrame(cm, columns=classes)

            clsreport = metrics.classification_report(y_true, y_pred, digits=4, output_dict=True)
            df = pd.DataFrame()
            for i in clsreport.keys():
                if i == "accuracy":
                    acc = clsreport[i]
                    continue
                df_ = pd.DataFrame(clsreport[i], index=[i])
                df = pd.concat([df, df_], axis=0)

            df.index = classes + df.index[-2:].tolist()            

            fig = plt.figure(figsize=(16, 10)) 
            ax1 = plt.subplot2grid((4, 4), (0, 0), rowspan=2, colspan=2)
            ax2 = plt.subplot2grid((4, 4), (0, 2), rowspan=2, colspan=2)
            ax3 = plt.subplot2grid((4, 4), (2, 1), rowspan=2, colspan=2)

            display_table(df, ax1, dtype=None, onlytable=True, treverse=True, tloc="center right", tscale=(0.75, 2.75), title="Accuracy, Precision, and Recall")
            plot_confusion_matrix(cm, fig=fig, ax=ax2, classes=classes, normalize=True)
            display_table(cmdf, ax3, dtype="cm", tscale=(1, 2), offset=toffset, bar_width=0.4, title="TP, TN, FP, FN")
            fig.tight_layout()
            
            if outd is not None:
                outf = "{}/perfeval.png".format(outd)
                if title_ is not None:
                    outf = "{}/perfeval_{}.png".format(outd, title_)

                logging.debug("tspredict".format(outf))
                plt.savefig(outf)
            else:
                plt.show()
            
            plt.close()
            
        elif "task" in kwarg.keys() and kwarg["task"] == "class2step":
            
            y_class = np.unique(np.append(np.unique(y_true), np.unique(y_pred)))
            classes = [classes[i] for i in y_class]
            
#             y_max = np.nanmax(y_true) 
#             if y_max < endpoints[-1]:
#                 endpoints[-1] = endpoints[-1] + 10
#             else:
#                 endpoints[-1] = y_max

            if np.isnan(kwarg["y2step"]).all():
                endpoints.append(np.nan)
            else:
                endpoints.append(np.nanmax(kwarg["y2step"]))

            steps = np.copy(y_pred)
            steps = [endpoints[i] for i in y_pred] 

            cm = metrics.confusion_matrix(y_true, y_pred)
            cmdf = pd.DataFrame(cm, columns=classes)

            clsreport = metrics.classification_report(y_true, y_pred, digits=4, output_dict=True)
            df = pd.DataFrame()
            for i in clsreport.keys():
                if i == "accuracy":
                    acc = clsreport[i]
                    continue
                df_ = pd.DataFrame(clsreport[i], index=[i])
                df = pd.concat([df, df_], axis=0)

            df.index = classes + df.index[-2:].tolist()            

            fig = plt.figure(figsize=(16, 10)) 
            ax1 = plt.subplot2grid((6, 4), (0, 0), rowspan=2, colspan=4)
            ax2 = plt.subplot2grid((6, 4), (2, 0), rowspan=2, colspan=2)
            ax3 = plt.subplot2grid((6, 4), (2, 2), rowspan=2, colspan=1)
            ax5 = plt.subplot2grid((6, 4), (2, 3), rowspan=2, colspan=1)
            plot_confusion_matrix(cm, fig=fig, ax=ax5, classes=classes, normalize=False, title="ConfusionMatrix")

            ax4 = plt.subplot2grid((6, 4), (4, 1), rowspan=2, colspan=2)


            ax1.plot(kwarg["y2step"], label="$y_{true}$") 
            ax1.step(np.arange(len(kwarg["y2step"])), steps, label="$\hat{y}_{pred}$")

            if (xposi_ is not None) and (xlabel_ is not None):
                assert len(xposi_) == len(xlabel_)
                ax1.set_xticks(xposi_)    
                ax1.set_xticklabels(xlabel_)    

            if title_ is not None:
                ax1.set_title(title_)

            ax1.legend(fontsize=14)
            display_table(df, ax2, dtype=None, onlytable=True, treverse=True, tloc="center right", tscale=(0.75, 2), title="Accuracy, Precision, and Recall")
            plot_confusion_matrix(cm, fig=fig, ax=ax3, classes=classes, normalize=True, title="ConfusionMatrix (Normalized)")
            display_table(cmdf, ax4, dtype="cm", tscale=(1, 1.5), offset=toffset, bar_width=0.4, title="TP, TN, FP, FN")
            
            fig.tight_layout()
#             fig.tight_layout(pad=0.1, w_pad=0.25, h_pad=0.25)

            
            if outd is not None:
                outf = "{}/perfeval.png".format(outd)
                if title_ is not None:
                    outf = "{}/perfeval_{}.png".format(outd, title_)

                logging.debug("{}".format(outf))
                plt.savefig(outf)
            else:
                plt.show()
            
            plt.close()
            
        else:
#             if "classify" in kwarg.keys():
#                 endpoints = kwarg["classify"]
#                 y_max = np.nanmax(y_true) 
#                 if y_max < endpoints[-1]:
#                     endpoints.append(endpoints[-1] + 10)
#                 else:
#                     endpoints.append(y_max)

#                 y_pred_ = np.copy(y_pred)
#                 y_pred = [endpoints[i] for i in y_pred] 

            ###  preprocessing    
            minv = min(np.nanmin(y_true), np.nanmin(y_pred))
            maxv = max(np.nanmax(y_true), np.nanmax(y_pred))
            
            residual_ = y_pred - y_true
            if np.isnan(residual_).all():
                logging.warning("whole residules is nan! tspredict stopped.")
                gc.collect()
                return
            
            nsize = len(residual_)
            rmse = (np.apply_along_axis(lambda x: x**2, 0, residual_).mean() / nsize)**0.5
            mu_  = residual_.mean()
            std_ = residual_.std(ddof=1)

            ###  normality test
            # the null hypothesis that the data was drawn from a normal distribution.
            stests, stestp = stats.shapiro(residual_)

            # the null hypothesis that a sample comes from a normal distribution.
            ntests, ntestp = stats.normaltest(residual_)

            descstat = "($\mu$, $\sigma$) = ({0:7.3f}, {1:7.3f})\n(shapiro, ntest) = ({2:7.3f}, {3:7.3f})".format(mu_, std_, stestp, ntestp)

            fig = plt.figure(figsize=(16, 10))

            ax1 = plt.subplot2grid(shape=(7, 4), loc=(0, 0), rowspan=3, colspan=4)

            ax1.plot(y_true, label="$y_{true}$") 
            if "classify" in kwarg.keys(): 
                ax1.step(np.arange(nsize), y_pred, label="$\hat{y}_{pred}$")
            else:
                ax1.plot(y_pred, label="$\hat{y}_{pred}$")

            if "steps" in kwarg.keys():
                ax1.plot(kwarg["steps"], label="classification")

            if (xposi_ is not None) and (xlabel_ is not None):
                assert len(xposi_) == len(xlabel_)
                ax1.set_xticks(xposi_)    
                ax1.set_xticklabels(xlabel_)    

            if title_ is not None:
                plt.title(title_)

            ax1.legend(fontsize=14)

            ax2 = plt.subplot2grid(shape=(7, 4), loc=(3, 0), rowspan=2, colspan=4)
            ax2.plot(residual_, label="residual", color="blue")
            ax2.plot(np.zeros(nsize), "--", color="red")

            if (xposi_ is not None) and (xlabel_ is not None):
                ax2.set_xticks(xposi_)    
                ax2.set_xticklabels(xlabel_)   

            ax2.legend(fontsize=14)

            ax2.text(nsize, min(residual_), "RMSE = {0:>7.3f}".format(rmse),
                     verticalalignment="bottom", horizontalalignment="right",
                     color="black", fontsize=14, bbox=props)

            ax3 = plt.subplot2grid(shape=(7, 4), loc=(5, 0), rowspan=2, colspan=2)
            n, bins, patches = ax3.hist(residual_, bins=30, density=True, stacked=True, label="residual", facecolor="g", alpha=0.75)            
            if np.nanmax(residual_) - np.nanmin(residual_) < 1e-3:
                resx = np.arange(-1, 1, 0.001)
            else:
                resx = np.arange(np.nanmin(residual_), np.nanmax(residual_), 0.001)
            resnpdf = norm(mu_, std_).pdf(resx)
            ax3.plot(resx, resnpdf, label="normal", color="red")
            ax3.text(bins[int(len(bins) * 0.001)], max(n) * 0.45, s=descstat, fontsize=12, bbox=props)
            ax3.legend(fontsize=14)

            ax4 = plt.subplot2grid(shape=(7, 4), loc=(5, 2), rowspan=2, colspan=2)
            ax4.scatter(y_true, y_pred, label="residual", color="skyblue")
            ax4.plot([minv, maxv], [minv, maxv], "--", color="red")

            fig.tight_layout()

            if outd is not None:
                outf = "{}/perfeval.png".format(outd)
                if title_ is not None:
                    outf = "{}/perfeval_{}.png".format(outd, title_)

                logging.debug("tspredict".format(outf))
                plt.savefig(outf)
            else:
                plt.show()

            plt.close()
        
        gc.collect()
        
        
    @staticmethod
    def tserror(ind, stnid, tperiod, fc="reg", tscale="H", tfmt="%Y%m%d%H", hsystem="00-23", lb=0, erroutd=None):
        
        if erroutd is not None:
            if not os.path.exists(erroutd):
                os.makedirs(erroutd)
        
        # get dtimes
        sdt = tperiod[0]
        edt = tperiod[1]

        htype = 0
        if hsystem == "01-24":
            htype = 1
            sdt_ = datetime.strptime(str(sdt)[0:8], "%Y%m%d")
            edt_ = datetime.strptime(str(edt)[0:8], "%Y%m%d")
        else:
            sdt_ = datetime.strptime(str(sdt), tfmt)
            edt_ = datetime.strptime(str(edt), tfmt)

        sttuple = sdt_.timetuple()
        ettuple = edt_.timetuple()

        syr = sttuple[0]
        eyr = ettuple[0]

        if tscale == "H":
            dtimes = wg.YYYYmmddHH0000(syr, eyr, htype)
            dtimes = [int(i / 10**4) for i in dtimes]
        elif tscale == "d":
            dtimes = wg.YYYYmmdd(syr, eyr)
        elif tscale == "m":
            dtimes = wg.YYYYmm(syr, eyr)

        sidx = np.where(np.array(dtimes) <= sdt)
        eidx = np.where(edt <= np.array(dtimes))

        if len(sidx[0]) == 0:
            sidx = 0
        else:
            sidx = max(sidx[0]) 

        if len(eidx[0]) == 0:
            eidx = len(dtimes) - 1
        else:
            eidx = min(eidx[0]) 

        nsize = eidx - sidx + 1    
        nstn = len(stnid)

#         print(dtimes[sidx], dtimes[eidx])
#         predf = ["{}/{}".format(ind, i) for i in os.listdir(ind)]

        # load predicttions
        darray = np.ndarray((nsize, nstn, 2))
        darray.fill(-999)
        idx = 0
        for dtidx, dtime in enumerate(dtimes):
            if dtidx < sidx or eidx < dtidx:
                continue

            fname = "{}/val_{}.csv".format(ind, dtime)
            if not os.path.exists(fname):
                logging.warning("{} doesn't exist.".format(fname))
                continue
            val = pd.read_csv(fname)
            val.columns = ["stnid", "obs", "est"]
            val.set_index("stnid", inplace=True)
            for sididx, sid in enumerate(stnid):
                darray[idx, sididx, 0] = val.loc[sid]["obs"]
                darray[idx, sididx, 1] = val.loc[sid]["est"]

#             print(idx, dtime)

            idx += 1
        
        # error check
        errors = np.zeros((len(stnid), 7))
        errors.fill(-999)
#         props = dict(boxstyle="round", facecolor="red", alpha=0.5)

        for sididx, sid in enumerate(stnid):
            arr = np.copy(darray[:, sididx, :])
            posi = np.where(np.isnan(arr).any(axis=1))
            arr = np.delete(arr, posi, axis=0)
            arr[np.where(arr < 0)] = np.nan
            posi = np.where(np.isnan(arr).any(axis=1))
            arr = np.delete(arr, posi, axis=0)

            if arr.size == 0:
                continue

            coldiff = np.apply_along_axis(lambda x: x[1] - x[0], 1, arr)  # residules

            errors[sididx, 0] = min(coldiff)
            errors[sididx, 1] = np.percentile(coldiff, 25)
            errors[sididx, 2] = np.percentile(coldiff, 50)
            errors[sididx, 3] = np.percentile(coldiff, 75)
            errors[sididx, 4] = max(coldiff)
            errors[sididx, 5] = coldiff.mean()
            errors[sididx, 6] = coldiff.std()


            if erroutd is not None:
                fig, ax = plt.subplots(2)

                descs = "$\mu$ = {0:6.3f}\n$\sigma$ = {1:6.3f}\n$p(0)$ = {2:6.3f}\n$p(50)$ = {3:6.3f}\n$p(100)$ = {4:6.3f}\n"
                descstat = descs.format(errors[sididx, 5], errors[sididx, 6], errors[sididx, 0], errors[sididx, 2], errors[sididx, 4])

                minv = arr.min()
                maxv = arr.max()

                # y-axis: n, x-axis: bins
                n, bins, patches = ax[0].hist(coldiff, bins=30, density=True, stacked=True, label="residual", facecolor="g", alpha=0.75)
                resx = np.arange(min(coldiff), max(coldiff), 0.001)
                resnpdf = norm(errors[sididx, 5], errors[sididx, 6]).pdf(resx)
                ax[0].plot(resx, resnpdf, label="normal", color="red")
                ax[0].text(bins[int(len(bins) * 0.75)], max(n) * 0.05, s=descstat, fontsize=12)

                ax[0].legend(fontsize=14)
                ax[0].set_title("Residules of LSTM {}".format(sid))

                ax[1].scatter(arr[:, 0], arr[:, 1], label="residual", color="skyblue")
                ax[1].plot([minv, maxv], [minv, maxv], "--", color="red")
                ax[1].set_xlabel("observation")
                ax[1].set_ylabel("predicition")
                ax[1].set_title("Scatter Plot of {}".format(sid))

                fig.tight_layout()

                plt.savefig("{}/err_{}.png".format(erroutd, sid))
                plt.close()
        
        errdf = pd.DataFrame(errors, columns=["p0", "p25", "p50", "p75", "p100", "mean", "std"])
        errdf.index = stnid
        if erroutd is not None:
            errdf.to_csv("{}/errors.csv".format(erroutd))

        return [darray, dtimes[sidx:(eidx + 1)], stnid, errdf]
    
    
    @staticmethod
    def tsqcinfo(darray, dtimes, stnid, errdf, multiplier, timestep, outd, vname="Temp", mname="LSTM"):
        for sididx, sid in enumerate(stnid):
            if not os.path.exists("{}/{}".format(outd, sid)):
                os.makedirs("{}/{}".format(outd, sid))

        Ym = str(dtimes[0])[0:6]
        sidx = 0
        eidx = 0
        qcinfo = []
        for idx, dtime in enumerate(dtimes):
            Ym_ = str(dtime)[0:6]

            if Ym_ == Ym:
                eidx = idx
            else:
                print(dtimes[sidx], dtimes[eidx])

                qcdt = np.array(dtimes[sidx:(eidx + 1)])

                for sididx, sid in enumerate(stnid):

                    obs = np.copy(darray[:, sididx, 0][sidx:(eidx + 1)])
                    est = np.copy(darray[:, sididx, 1][sidx:(eidx + 1)])
                    upper = est + multiplier * errdf.loc[sid]["std"]
                    lower = est - multiplier * errdf.loc[sid]["std"]

                    nsize = len(obs)

                    if sidx == 0:  # the first timestep estimations are not reliable (# of input < timestep)
                        obs[0:(timestep + 1)] = np.nan

                    if np.isnan(obs).all():
                        logging.warning("{} is missing for {} ({} - {}), all nan".format(vname, sid, dtimes[sidx], dtimes[eidx]))
                        continue
                    else:
                        missings_ = np.where(np.isnan(obs))[0]
                        missings = []
                        for i in missings_:
                            for j in range(timestep + 1):
                                if i + j >= nsize:
                                    break
                                missings.append(i + j)
                        missings = list(set(missings))
                        obs[missings] = np.nan
                        if np.isnan(obs).all():
                            logging.warning("{} is missing for {} ({} - {}) after data processing".format(vname, sid, dtimes[sidx], dtimes[eidx]))
                            continue

                    est     = est[~np.isnan(obs)]
                    upper   = upper[~np.isnan(obs)]
                    lower   = lower[~np.isnan(obs)]
                    qcdtime = qcdt[~np.isnan(obs)]
                    obs     = obs[~np.isnan(obs)]

                    if len(obs) <= 8:
                        logging.warning("{} is missing for {} ({} - {}), nsize = {}, len(obs) = {}".format(vname, sid, dtimes[sidx], dtimes[eidx], nsize, len(obs)))
                        continue

                    gtu_posi = np.where(obs > upper)
                    ltl_posi = np.where(obs < lower)

                    text_on_plt = ""
                    gtupper = None
                    ltlower = None
                    if len(gtu_posi[0]) > 0:
                        gtupper = obs[gtu_posi]  # great than upper bound 
                        for i in gtu_posi[0]:
                            qcinfo.append([sid, qcdtime[i], obs[i], est[i], lower[i], upper[i]])

                    if len(ltl_posi[0]) > 0:
                        ltlower = obs[ltl_posi]  # less than lower bound
                        for i in ltl_posi[0]:
                            qcinfo.append([sid, qcdtime[i], obs[i], est[i], lower[i], upper[i]])

                    fig, ax = plt.subplots(figsize=(22, 10))
                    ax.plot(est, label="est")
                    ax.plot(upper, color="blue", linestyle="dashed", label="upper")
                    ax.plot(lower, color="royalblue", linestyle="dashed", label="lower")
                    ax.plot(obs, color="black", label="obs")

                    if gtupper is not None:
                        ax.scatter(gtu_posi[0], gtupper, color="green", label="suspect (> upper)")

                    if ltlower is not None:
                        ax.scatter(ltl_posi[0], ltlower, color="purple", label="suspect (< lower)")

                    xposi = np.arange(0, len(obs), int(len(obs) / 8))
                    xposi = np.append(xposi, len(obs) - 1)
                    xlabel = qcdtime[xposi]

                    ax.set_xticks(xposi)    
                    ax.set_xticklabels(xlabel, rotation=25, fontsize=14)
        #             logging.info("{} QC for {} ({} - {}, {})".format(vname, sid, dtimes[sidx], dtimes[eidx], mname))
                    ax.set_title("{} QC for {} ({} - {}, {})".format(vname, sid, dtimes[sidx], dtimes[eidx], mname), fontsize=20)
                    ax.legend(fontsize=14)

                    plt.yticks(fontsize=14)
                    plt.xlim(-10, len(obs) + 10)            
                    plt.tight_layout()
                    plt.savefig("{}/{}/{}_{}_{}_{}.png".format(outd, sid, vname, sid, dtimes[sidx], dtimes[eidx]))
                    plt.close()

                Ym = Ym_
                sidx = idx                    
                
                gc.collect()
                
        qcout = copy.deepcopy(qcinfo)
        qcout.sort()            
            
        if len(qcout) <= 0:
            return 0
            
        if not os.path.exists("{}/QCInfo".format(outd)):
            os.makedirs("{}/QCInfo".format(outd))

        ids = []
        hfmt = "{0:6s} {1:>12s} {2:>12s} {3:>12s} {4:>12s} {5:>12s}\n"  # header
        ffmt = "{0:6s} {1:12d} {2:12.3f} {3:12.3f} {4:12.3f} {5:12.3f}\n"  
        wid = qcout[0][0] 
        fid = open("{}/QCInfo/QC_{}.txt".format(outd, wid), "w")
        fid.write(hfmt.format("stnId", "datetime", "obs", "est", "lower", "upper"))
        
        for idx, qc in enumerate(qcout):
#             if idx == 0:
#                 wid = qc[0]
#                 ids.append(wid)
#                 fid = open("{}/QCInfo/QC_{}.txt".format(outd, wid), "w")
#                 fid.write(hfmt.format("stnId", "datetime", "obs", "est", "lower", "upper"))
#             else:
            if qc[0] != wid:
                fid.close()
                wid = qc[0]
                fid = open("{}/QCInfo/QC_{}.txt".format(outd, wid), "w")
                fid.write(hfmt.format("stnId", "datetime", "obs", "est", "lower", "upper"))
                if wid in ids:
                    logging.warning("qcinfo sort failed (got duplicated stnid)")
                ids.append(wid)

            fid.write(ffmt.format(qc[0], qc[1], qc[2], qc[3], qc[4], qc[5]))

        fid.close()
    



In [ ]:
ids = []
hfmt = "{0:6s} {1:>12s} {2:>12s} {3:>12s} {4:>12s} {5:>12s}\n"  # header
ffmt = "{0:6s} {1:12d} {2:12.3f} {3:12.3f} {4:12.3f} {5:12.3f}\n"  
wid = qcout[0][0] 
fid = open("{}/QCInfo/QC_{}.txt".format(outd, wid), "w")
fid.write(hfmt.format("stnId", "datetime", "obs", "est", "lower", "upper"))

for idx, qc in enumerate(qcout):
#             if idx == 0:
#                 wid = qc[0]
#                 ids.append(wid)
#                 fid = open("{}/QCInfo/QC_{}.txt".format(outd, wid), "w")
#                 fid.write(hfmt.format("stnId", "datetime", "obs", "est", "lower", "upper"))
#             else:
    if qc[0] != wid:
        fid.close()
        wid = qc[0]
        fid = open("{}/QCInfo/QC_{}.txt".format(outd, wid), "w")
        fid.write(hfmt.format("stnId", "datetime", "obs", "est", "lower", "upper"))
        if wid in ids:
            logging.warning("qcinfo sort failed (got duplicated stnid)")
        ids.append(wid)

    fid.write(ffmt.format(qc[0], qc[1], qc[2], qc[3], qc[4], qc[5]))

fid.close()
    

In [ ]:
ind      = "/NAS-129/users1/T1/DATA/RNN/QC/stackedLSTM_MAE_MinMax_HourlyTemp/pred"
erroutd  = "/NAS-129/users1/T1/DATA/RNN/QC/stackedLSTM_MAE_MinMax_HourlyTemp/errcheck"
tperiod  = [2016010101, 2019123124]
# tperiod  = [2016110101, 2017013124]

tscale   = "H"
tfmt     = "%Y%m%d%H"
hsystem  = "01-24"
lb       = -20

stninfo  = "/home/yuzhe/CODE/ProgramT1/GRDTools/SRC/RES/GI/1500_decode_stationlist_without_space.txt"
stnid    = textloader.get_id(stninfo)

In [ ]:
darray, dtimes, stnid, errdf = perfeval.tserror(ind, stnid, tperiod, tscale='H', tfmt=tfmt, hsystem=hsystem, lb=lb, erroutd=erroutd)

In [ ]:
multiplier = 3.0
timestep = 6
outd = "/NAS-129/users1/T1/DATA/RNN/QC/stackedLSTM_5MAE_MinMax_HourlyTemp/qcinfo"

perfeval.tsqcinfo(darray, dtimes, stnid, errdf, multiplier, timestep, outd, vname="Temp", mname="LSTM")


# Rate of checking

In [89]:
errf = "/NAS-129/users1/T1/DATA/RNN/QC/stackedLSTM_MAE_MinMax_HourlyTemp/errcheck/errors.csv"
errors = pd.read_csv(errf)
errors.columns = ["stnId", "p0", "p25", "p50", "p75", "p100", "mean", "std"]
errors.set_index("stnId", inplace=True)
errors[errors["std"] > 0].shape



(401, 7)

In [92]:
outd = "/NAS-129/users1/T1/DATA/RNN/QC/stackedLSTM_MAE_MinMax_HourlyTemp/qcinfo/QCInfo"
qcfs = os.listdir(outd)
df = pd.DataFrame()
for qcf in qcfs:
    fname = "{}/{}".format(outd, qcf)
    if fname[-3:] != "txt":
        continue
#     print(fname)
    df_ = pd.read_csv("/NAS-129/users1/T1/DATA/RNN/QC/stackedLSTM_MAE_MinMax_HourlyTemp/qcinfo/QCInfo/QC_467400.txt", sep="\s+")
    df = pd.concat([df, df_])
    
df.shape
# a = pd.read_csv("/NAS-129/users1/T1/DATA/RNN/QC/stackedLSTM_MAE_MinMax_HourlyTemp/qcinfo/QCInfo/QC_467400.txt", sep="\s+")
# b = pd.read_csv("/NAS-129/users1/T1/DATA/RNN/QC/stackedLSTM_MAE_MinMax_HourlyTemp/qcinfo/QCInfo/QC_466920.txt", sep="\s+")

(141954, 6)

In [98]:
a = 401 * 2 * 365 * 24 + 401 * 366 * 24
# a = 401 * 3 * 365 * 24

b = 141954
b / a 

0.013458029197080291

In [ ]:
outd = "LSTMQC"


for sididx, sid in enumerate(stnid):
    if not os.path.exists("{}/{}".format(outd, sid)):
        os.makedirs("{}/{}".format(outd, sid))

vname = "Temp"
mname = "LSTM"
multiplier = 3.0
Ym = str(dtimes[0])[0:6]
sidx = 0
eidx = 0
qcinfo = []
for idx, dtime in enumerate(dtimes):
    Ym_ = str(dtime)[0:6]

    if Ym_ == Ym:
        eidx = idx
    else:
        print(dtimes[sidx], dtimes[eidx])
        
        qcdt = np.array(dtimes[sidx:(eidx + 1)])
        
        for sididx, sid in enumerate(stnid):

#         for sididx, sid in enumerate(stnid[0:10]):
            obs = np.copy(darray[:, sididx, 0][sidx:(eidx + 1)])
            est = np.copy(darray[:, sididx, 1][sidx:(eidx + 1)])
            upper = est + multiplier * errdf.loc[sid]["std"]
            lower = est - multiplier * errdf.loc[sid]["std"]
            
            nsize = len(obs)
            
            if sidx == 0:  # the first timestep estimations are not reliable (# of input < timestep)
                obs[0:(timestep + 1)] = np.nan
            
            if np.isnan(obs).all():
                logging.warning("{} is missing for {} ({} - {}), all nan".format(vname, sid, dtimes[sidx], dtimes[eidx]))
                continue
            else:
                missings_ = np.where(np.isnan(obs))[0]
                missings = []
                for i in missings_:
                    for j in range(timestep + 1):
                        if i + j >= nsize:
                            break
                        missings.append(i + j)
                missings = list(set(missings))
                obs[missings] = np.nan
                if np.isnan(obs).all():
                    logging.warning("{} is missing for {} ({} - {}) after data processing".format(vname, sid, dtimes[sidx], dtimes[eidx]))
                    continue
            
            est     = est[~np.isnan(obs)]
            upper   = upper[~np.isnan(obs)]
            lower   = lower[~np.isnan(obs)]
            qcdtime = qcdt[~np.isnan(obs)]
            obs     = obs[~np.isnan(obs)]
            
            if len(obs) <= 8:
                logging.warning("{} is missing for {} ({} - {}), nsize = {}, len(obs) = {}".format(vname, sid, dtimes[sidx], dtimes[eidx], nsize, len(obs)))
                continue
            
            gtu_posi = np.where(obs > upper)
            ltl_posi = np.where(obs < lower)
            
            text_on_plt = ""
            gtupper = None
            ltlower = None
            if len(gtu_posi[0]) > 0:
                gtupper = obs[gtu_posi]  # great than upper bound 
                for i in gtu_posi[0]:
                    qcinfo.append([sid, qcdtime[i], obs[i], est[i], lower[i], upper[i]])
            
            if len(ltl_posi[0]) > 0:
                ltlower = obs[ltl_posi]  # less than lower bound
                for i in ltl_posi[0]:
                    qcinfo.append([sid, qcdtime[i], obs[i], est[i], lower[i], upper[i]])

            fig, ax = plt.subplots(figsize=(22, 10))
            ax.plot(est, label="est")
            ax.plot(upper, color="blue", linestyle="dashed", label="upper")
            ax.plot(lower, color="royalblue", linestyle="dashed", label="lower")
            ax.plot(obs, color="black", label="obs")
            
            if gtupper is not None:
                ax.scatter(gtu_posi[0], gtupper, color="green", label="suspect (> upper)")
                
            if ltlower is not None:
                ax.scatter(ltl_posi[0], ltlower, color="purple", label="suspect (< lower)")
           
            xposi = np.arange(0, len(obs), int(len(obs) / 8))
            xposi = np.append(xposi, len(obs) - 1)
            xlabel = qcdtime[xposi]
#             print(xposi, xlabel)
            ax.set_xticks(xposi)    
            ax.set_xticklabels(xlabel, rotation=25, fontsize=14)
#             logging.info("{} QC for {} ({} - {}, {})".format(vname, sid, dtimes[sidx], dtimes[eidx], mname))
            ax.set_title("{} QC for {} ({} - {}, {})".format(vname, sid, dtimes[sidx], dtimes[eidx], mname), fontsize=20)
            ax.legend(fontsize=14)
            
            plt.yticks(fontsize=14)
            plt.xlim(-10, len(obs) + 10)            
            plt.tight_layout()
            
#             plt.show()
            plt.savefig("{}/{}/{}_{}_{}_{}.png".format(outd, sid, vname, sid, dtimes[sidx], dtimes[eidx]))
            plt.close()

        Ym = Ym_
        sidx = idx

print(qcinfo)

In [ ]:
qcout = copy.deepcopy(qcinfo)
qcout.sort()

In [ ]:
if not os.path.exists("{}/QCInfo".format(outd)):
    os.makedirs("{}/QCInfo".format(outd))

wid = None
ididx = 1
ids = []
hfmt = "{0:6s} {1:>12s} {2:>12s} {3:>12s} {4:>12s} {5:>12s}\n"  # header
ffmt = "{0:6s} {1:12d} {2:12.3f} {3:12.3f} {4:12.3f} {5:12.3f}\n"  
for idx, qc in enumerate(qcout):
    if idx == 0:
        wid = qc[0]
        ids.append(wid)
        fid = open("{}/QCInfo/QC_{}.txt".format(outd, wid), "w")
        fid.write(hfmt.format("stnId", "datetime", "obs", "est", "lower", "upper"))
    else:
        if qc[0] != wid:
            fid.close()
            fid = open("{}/QCInfo/QC_{}.txt".format(outd, wid), "w")
            fid.write(hfmt.format("stnId", "datetime", "obs", "est", "lower", "upper"))
            ididx += 1
            wid = qc[0]
            if wid in ids:
                logging.warning("qcinfo sort failed (got duplicated stnid)")
            ids.append(wid)
            
    fid.write(ffmt.format(qc[0], qc[1], qc[2], qc[3], qc[4], qc[5]))
    
fid.close()
    
#     print(i[0])



In [ ]:
multiplier = 3.0
for sidx, sid in enumerate(stnid[0:10]):
    obs = darray[:, sidx, 0]
    est = darray[:, sidx, 1]
    upper = est + multiplier * errdf.loc[sid]["std"]
    lower = est - multiplier * errdf.loc[sid]["std"]

    fig, ax = plt.subplots()
    ax.plot(est)
#     ax.plot(upper, color="blue")
#     ax.plot(lower, color="blue")

    ax.plot(obs, color="green")
    plt.show()
    plt.close()

In [ ]:
darray, dtimes, stnid, errdf = perfeval.tserror(ind, stnid, tperiod, tscale='H', tfmt=tfmt, hsystem=hsystem, lb=lb, erroutd=erroutd)

In [ ]:
sdt = tperiod[0]
edt = tperiod[1]

htype = 0
if hsystem == "01-24":
    htype = 1
    sdt_ = datetime.strptime(str(sdt)[0:8], "%Y%m%d")
    edt_ = datetime.strptime(str(edt)[0:8], "%Y%m%d")
else:
    sdt_ = datetime.strptime(str(sdt), tfmt)
    edt_ = datetime.strptime(str(edt), tfmt)

sttuple = sdt_.timetuple()
ettuple = edt_.timetuple()

syr = sttuple[0]
eyr = ettuple[0]

if tscale == "H":
    dtimes = wg.YYYYmmddHH0000(syr, eyr, htype)
    dtimes = [int(i / 10**4) for i in dtimes]
elif tscale == "d":
    dtimes = wg.YYYYmmdd(syr, eyr)
elif tscale == "m":
    dtimes = wg.YYYYmm(syr, eyr)
    
print(sdt, edt)
sidx = np.where(np.array(dtimes) <= sdt)
eidx = np.where(edt <= np.array(dtimes))
print(sidx, eidx)

if len(sidx[0]) == 0:
    sidx = 0
else:
    sidx = max(sidx[0]) 
    
if len(eidx[0]) == 0:
    eidx = len(dtimes) - 1
else:
    eidx = min(eidx[0]) 
    
nsize = eidx - sidx + 1    
nstn = len(stnid)

predf = ["{}/{}".format(ind, i) for i in os.listdir(ind)]

darray = np.ndarray((nsize, nstn, 2))
darray.fill(-999)
for idx, dtime in enumerate(dtimes):
    if idx < sidx or eidx < idx:
        continue
       
    fname = "{}/val_{}.csv".format(ind, dtime)
    if not os.path.exists(fname):
        logging.warning("{} doesn't exist.".format(fname))
        continue
    val = pd.read_csv(fname)
    val.columns = ["stnid", "obs", "est"]
    val.set_index("stnid", inplace=True)
    for sidx, sid in enumerate(stnid):
        darray[idx, sidx, 0] = val.loc[sid]["obs"]
        darray[idx, sidx, 1] = val.loc[sid]["est"]

# reobj = re.compile(pattern=".*_([0-9]*).*csv$")
# dtime = reobj.match(predf[2]).group(1)

In [ ]:

errors = np.zeros((len(stnid), 7))
errors.fill(-999)
props = dict(boxstyle="round", facecolor="red", alpha=0.5)

if erroutd is not None:
    if not os.path.exists(erroutd):
        os.makedirs(erroutd)
    
for sidx, sid in enumerate(stnid):
    arr = np.copy(darray[:, sidx, :])
    posi = np.where(np.isnan(arr).any(axis=1))
    arr = np.delete(arr, posi, axis=0)
    arr[np.where(arr < 0)] = np.nan
    posi = np.where(np.isnan(arr).any(axis=1))
    arr = np.delete(arr, posi, axis=0)

    if arr.size == 0:
        continue
        
#     print(arr.shape)    
    coldiff = np.apply_along_axis(lambda x: x[1] - x[0], 1, arr)
#     err = coldiff.mean()
    
        
    errors[sidx, 0] = min(coldiff)
    errors[sidx, 1] = np.percentile(coldiff, 25)
    errors[sidx, 2] = np.percentile(coldiff, 50)
    errors[sidx, 3] = np.percentile(coldiff, 75)
    errors[sidx, 4] = max(coldiff)
    errors[sidx, 5] = coldiff.mean()
    errors[sidx, 6] = coldiff.std()

    fig, ax = plt.subplots(2)
    

#     descs = "$\mu$ = {0:6.3f}\n$\sigma$ = {1:6.3f}\n$p_0$ = {2:6.3f}\n$p_{25}$ = {3:6.3f}\n$p_{50}$ = {4:6.3f}\n$p_{75}$ = {5:6.3f}\n$p_{100}$ = {6:6.3f}\n"
    descs = "$\mu$ = {0:6.3f}\n$\sigma$ = {1:6.3f}\n$p(0)$ = {2:6.3f}\n$p(50)$ = {3:6.3f}\n$p(100)$ = {4:6.3f}\n"

#     descstat = descs.format(errors[sidx, 5], errors[sidx, 6], errors[sidx, 0], errors[sidx, 1], errors[sidx, 2], errors[sidx, 3], errors[sidx, 4])
    descstat = descs.format(errors[sidx, 5], errors[sidx, 6], errors[sidx, 0], errors[sidx, 2], errors[sidx, 4])

    minv = arr.min()
    maxv = arr.max()

#     ax2.text(nsize, min(residual_), "RMSE = {0:>7.3f}".format(rmse),
#              verticalalignment="bottom", horizontalalignment="right",
#              color="black", fontsize=14, bbox=props)

#     ax3 = plt.subplot2grid(shape=(7, 4), loc=(5, 0), rowspan=2, colspan=2)

    # y-axis: n, x-axis: bins
    n, bins, patches = ax[0].hist(coldiff, bins=30, density=True, stacked=True, label="residual", facecolor="g", alpha=0.75)
    resx = np.arange(min(coldiff), max(coldiff), 0.001)
    resnpdf = norm(errors[sidx, 5], errors[sidx, 6]).pdf(resx)
    ax[0].plot(resx, resnpdf, label="normal", color="red")
#     ax[0].text(bins[int(len(bins) * 0.75)], max(n) * 0.45, s=descstat, fontsize=12, bbox=props)
    ax[0].text(bins[int(len(bins) * 0.75)], max(n) * 0.05, s=descstat, fontsize=12)

    ax[0].legend(fontsize=14)
    ax[0].set_title("Residules of LSTM {}".format(sid))

#     ax4 = plt.subplot2grid(shape=(7, 4), loc=(5, 2), rowspan=2, colspan=2)
    ax[1].scatter(arr[:, 0], arr[:, 1], label="residual", color="skyblue")
    ax[1].plot([minv, maxv], [minv, maxv], "--", color="red")
    ax[1].set_xlabel("observation")
    ax[1].set_ylabel("predicition")
    ax[1].set_title("Scatter Plot of {}".format(sid))


#     ax.hist(coldiff, bins=30)
# #     ax.text()
    
    fig.tight_layout()

    
#     plt.show()
    plt.savefig("{}/err_{}.png".format(erroutd, sid))
    plt.close()


    
# for sidx, sid in enumerate(stnid[0:10]):
#     errors = np.zeros((len(stnid), 7))


In [ ]:
errdf = pd.DataFrame(errors, columns=["p0", "p25", "p50", "p75", "p100", "mean", "std"])
errdf.index = stnid
errdf.to_csv("{}/errors.csv".format(erroutd))


In [ ]:
inputf = "/NAS-129/users1/T1/DATA/RNN/QC/bidirectionalLSTM_MAEpCCE_MinMax/pred/466920.csv"

df = pd.read_csv(inputf, sep=",")
df.columns = ["dtime", "y_true", "y_pred"]
y_true = df["y_true"].values
y_pred = df["y_pred"].values

print(len(y_true))
mask_n = 2


n1 = 19000
n2 = 19500
missposi = np.where(np.isnan(y_true))[0]
for i in missposi:
    for j in range(mask_n):
        y_true[i - j - 1] = np.nan
        if i + j + 1 >= len(y_true):
            continue
        y_true[i + j + 1] = np.nan
plt.plot(y_true)

y_pred[np.where(np.isnan(y_true))] = np.nan
plt.plot(y_pred)


In [ ]:

if __name__ == "__main__":

    a = getClasses([0.1, 5, 10, 20])
    # fig = plt.figure()
    ax = plt.subplot2grid((1, 1), (0, 0), colspan=1, rowspan=1)
    ax.text(0.1, 0.1, a[0])
    ax.text(0.12, 0.12, a[1])
    ax.text(0.25, 0.25, a[2])
    ax.text(0.37, 0.37, a[3])
    ax.text(0.49, 0.49, a[4])
    
    
    n = 200
    y_true = np.random.random_sample(n)
    y_pred = np.random.random_sample(n)

    y_true[-1] = 10
    y_true[0] = 43
    perfeval.tspredict(y_true, y_pred)

    
    
    classify = [0.1, 5, 10, 20]
    if np.nanmax(y_true) < classify[-1]:
        classify.append(classify[-1] + 10)
    else:
        classify.append(np.nanmax(y_true))
        
    y_pred = np.random.choice([0, 1, 2, 3, 4], n, replace=True)
#     y_pred = [classify[i] for i in y_pred]
    
    perfeval.tspredict(y_true, y_pred, classify=classify)


In [ ]:
if __name__ == "__main__":
    n = 10000
    y_true = np.random.choice([0, 1, 2, 3, 4], n, replace=True)
#     y_true = np.random.choice([0, 1], n, replace=True)

    y_pred = np.random.choice([0, 1, 2, 3, 4], n, replace=True)
#     y_pred = np.random.choice([0, 1, 2, 3], n, replace=True)

    classify = [0.1, 5, 10, 20, 30]
    classes = getClasses(classify)

    y_class = np.unique(np.append(np.unique(y_true), np.unique(y_pred)))
    classes = [classes[i] for i in y_class]

    # confusion matrix
    cm = metrics.confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, classes=classes)

    # cls report
    clsreport = metrics.classification_report(y_true, y_pred, digits=4, output_dict=True)
    print(metrics.classification_report(y_true, y_pred, digits=4))
    df = pd.DataFrame()
    for i in clsreport.keys():
        if i == "accuracy":
            acc = clsreport[i]
            print("acc: ", acc)
            continue
        df_ = pd.DataFrame(clsreport[i], index=[i])
    #     print(df_)
        df = pd.concat([df, df_], axis=0)

    df.index = classes + df.index[-2:].tolist()
    print(df)

    y2step = np.random.rand(n) * 10
    perfeval.tspredict(y_true, y_pred, task="class2step", classify=classify, toffset=0.1, y2step=y2step)


In [ ]:
fig = plt.figure(figsize=(16, 10)) 
ax1 = plt.subplot2grid((4, 4), (0, 0), rowspan=2, colspan=2)
ax2 = plt.subplot2grid((4, 4), (0, 2), rowspan=2, colspan=2)
ax3 = plt.subplot2grid((4, 4), (2, 1), rowspan=2, colspan=2)

onlytable=False
onlytable=True

dtype = "df"
offset=0.3
bar_width =0.4
treverse=True
title="Accuracy, Precision, and Recall"
tscale=(0.5, 1)
tloc="bottom"
tloc="center right"


display_table(df, ax1, dtype=None, onlytable=onlytable, treverse=treverse, tloc=tloc, tscale=(0.75, 3), offset=0.3, bar_width=0.4, title=title)
plot_confusion_matrix(cm, fig=fig, ax=ax2, classes=["fdasdfaf", "afdasfaf", "adfadfafd"], normalize=True)

cmdf = pd.DataFrame(cm, columns=["cls1", "cls2", "cls3"])
display_table(cmdf, ax3, dtype="cm", onlytable=False, treverse=False, tloc="bottom", tscale=(1, 2), offset=0.3, bar_width=0.2, title="TP, TN, FP, FN")
fig.tight_layout()


In [ ]:
import itertools

def plot_confusion_matrix(cm, 
                          classes,
                          ax=None,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        cm: confusion matrix whose indexes are true labels and columns are predicted labels
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # np.newaxis: increse one dimension
        logging.info("Normalized confusion matrix")
    else:
        logging.info('Confusion matrix, without normalization')


    if ax is None:
        ax = plt
    
    ax.imshow(cm, interpolation='nearest', cmap=cmap)
    try:
        ax.title(title)
    except:
        ax.set_title(title)
        
    ax.colorbar()
    
    tick_marks = np.arange(len(classes))
    ax.set_xticks(tick_marks, classes, rotation=45)
    ax.set_yticks(tick_marks, classes)
    ax.grid(b=False)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        ax.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    ax.ylabel('True label')
    ax.xlabel('Predicted label')
    ax.tight_layout()
 

In [ ]:
y_class = np.random.choice([0, 1, 4], 100)

y_class_ = np.copy(y_class)
# print(y_class)
y_class = pd.DataFrame({'y_class': y_class})
# y_class = pd.get_dummies(y_class['y_class']).values
y_class = pd.get_dummies(y_class['y_class'])
# print(type(y_class))
# print(y_class)
# y_class.T.reindex([0, 1, 2, 3, 4, 5]).fillna(0).T.values
# y_class.T.reindex([0, 1, 2, 3, 4, 5]).T.fillna(0)

# y_class = np.argmax(y_class, axis=1)

# ROC Curve

In [ ]:
y = np.random.choice([0, 1], 100)
scores = np.random.rand(100)
fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=1)
fpr
tpr
thresholds
plt.plot(fpr, tpr)

# 3D classification

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def randrange(n, vmin, vmax):
    '''
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    '''
    return (vmax - vmin)*np.random.rand(n) + vmin

fig = plt.figure()
ax = fig.add_subplot(231, projection='3d')

n = 100

# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].
# for c, m, zlow, zhigh in [('r', 'o', -50, -25), ('b', '^', -30, -5)]:
#     xs = randrange(n, 23, 32)
#     ys = randrange(n, 0, 100)
#     zs = randrange(n, zlow, zhigh)
#     ax.scatter(xs, ys, zs, c=c, marker=m)

Temp = np.random.rand(100) * 5
RH = np.random.rand(100)
Precp = np.random.rand(100)

ax.scatter(xs=Temp, ys=RH, zs=Precp, c="r", marker="^")


ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

ax3 = fig.add_subplot(233)
ax5 = fig.add_subplot(235, projection="3d")
Temp = np.random.rand(100) * 5
RH = np.random.rand(100)
Precp = np.random.rand(100)

ax5.scatter(xs=Temp, ys=RH, zs=Precp, c="r", marker="^", label="cls1")
Temp = np.random.rand(100) * 5
RH = np.random.rand(100)
Precp = np.random.rand(100)

ax5.scatter(xs=Temp, ys=RH, zs=Precp, c="b", marker="o", label="cls2")
ax5.legend()

plt.tight_layout()
plt.show()
